In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Store Part I results into DataFrame

Load the csv exported in Part I to a DataFrame

In [3]:
# Read Weather CSV

weather_data_df = pd.read_csv("output_data/cities.csv")

In [4]:
weather_data_df

,City ID,City Name,Country,City Latitude,City Longitude,City Temperature,City Max Temperature,City Humidity,City Cloudiness,City Wind Speed
0,3450909,Rondonópolis,BR,-16.4708,-54.6356,92.28,92.28,23,100,5.17
1,1650232,Banjar,ID,-8.1900,114.9675,80.60,80.60,94,20,6.91
2,4020109,Atuona,PF,-9.8000,-139.0333,78.64,78.64,75,27,7.23
3,1154689,Ko Samui,TH,9.5357,99.9357,83.46,83.46,71,100,3.89
4,770425,Janów Lubelski,PL,50.7070,22.4104,60.98,63.00,64,100,5.99
...,...,...,...,...,...,...,...,...,...,...
552,1636022,Martapura,ID,-3.4167,114.8500,77.00,77.00,100,90,1.14
553,2297834,Mpraeso,GH,6.5932,-0.7346,86.95,86.95,49,100,5.66
554,3693584,Pimentel,PE,-6.8367,-79.9342,70.84,75.20,60,0,8.05
555,847634,Rypefjord,NO,70.6333,23.6667,28.40,28.40,80,75,16.11


# Humidity Heatmap

Configure gmaps.Use the Lat and Lng as locations and Humidity as the weight.Add Heatmap layer to map.

In [6]:
# Configure gmaps with API key
# https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html

gmaps.configure(api_key = g_key)

locations = weather_data_df[["City Latitude","City Longitude"]]
humidity = weather_data_df["City Humidity"]

fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=humidity))
fig

Figure(layout=FigureLayout(height='420px'))

# Create new DataFrame fitting weather criteria

A max temperature lower than 80 degrees but higher than 70.

Wind speed less than 10 mph.

Zero cloudiness.

Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

In [7]:
# Vacation Data

vacation_data_df = weather_data_df.loc[(weather_data_df["City Max Temperature"] < 80) &
                                       (weather_data_df["City Max Temperature"] > 70) &
                                       (weather_data_df["City Wind Speed"] < 10) &
                                       (weather_data_df["City Cloudiness"] == 0)]

vacation_data_df

,City ID,City Name,Country,City Latitude,City Longitude,City Temperature,City Max Temperature,City Humidity,City Cloudiness,City Wind Speed
37,3430863,Mar del Plata,AR,-38.0023,-57.5575,73.40,73.40,60,0,6.91
138,935215,Saint-Philippe,RE,-21.3585,55.7679,71.35,75.20,83,0,4.61
197,2063042,Port Hedland,AU,-20.3167,118.5667,71.60,71.60,73,0,3.44
204,4033543,Teahupoo,PF,-17.8333,-149.2667,75.20,75.20,88,0,4.61
214,3840300,Presidencia Roque Sáenz Peña,AR,-26.7852,-60.4388,71.01,71.01,53,0,1.99
280,4034438,Haapiti,PF,-17.5667,-149.8667,75.20,75.20,88,0,4.61
308,3437920,Itakyry,PY,-24.9333,-55.2167,73.40,73.40,46,0,6.91
352,1529376,Korla,CN,41.7597,86.1469,73.58,73.58,15,0,3.18
360,1521379,Lenger,KZ,42.1830,69.8839,75.20,75.20,33,0,8.95
406,1185095,Teknāf,BD,20.8624,92.3058,79.23,79.23,83,0,1.41


In [8]:
cities_criteria = len(vacation_data_df["City Name"])

In [9]:
print(f"There are {cities_criteria} cities that are within the criteria")

There are 15 cities that are within the criteria


# Hotel Map

- Store into variable named hotel_df.
- Add a "Hotel Name" column to the DataFrame.
- Set parameters to search for hotels with 5000 meters.
- Hit the Google Places API for each city's coordinates.
- Store the first Hotel result into the DataFrame.
- Plot markers on top of the heatmap.

In [10]:
# Store into a variable named hotel_df

hotel_df = vacation_data_df.copy()

In [11]:
# Add a "Hotel Name" column to the DataFrame

hotel_df["Hotel Name"] = ""

In [12]:
hotel_df

,City ID,City Name,Country,City Latitude,City Longitude,City Temperature,City Max Temperature,City Humidity,City Cloudiness,City Wind Speed,Hotel Name
37,3430863,Mar del Plata,AR,-38.0023,-57.5575,73.40,73.40,60,0,6.91,
138,935215,Saint-Philippe,RE,-21.3585,55.7679,71.35,75.20,83,0,4.61,
197,2063042,Port Hedland,AU,-20.3167,118.5667,71.60,71.60,73,0,3.44,
204,4033543,Teahupoo,PF,-17.8333,-149.2667,75.20,75.20,88,0,4.61,
214,3840300,Presidencia Roque Sáenz Peña,AR,-26.7852,-60.4388,71.01,71.01,53,0,1.99,
280,4034438,Haapiti,PF,-17.5667,-149.8667,75.20,75.20,88,0,4.61,
308,3437920,Itakyry,PY,-24.9333,-55.2167,73.40,73.40,46,0,6.91,
352,1529376,Korla,CN,41.7597,86.1469,73.58,73.58,15,0,3.18,
360,1521379,Lenger,KZ,42.1830,69.8839,75.20,75.20,33,0,8.95,
406,1185095,Teknāf,BD,20.8624,92.3058,79.23,79.23,83,0,1.41,


In [13]:
# Set parameters to search for hotels with 5000 meters

target_search = "Hotel"
target_radius = "5000"
target_type = "Hotel"

# Set up a parameters dicitonary
params = {
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key":g_key   
}

In [14]:
# Hit the Google Places API for each city's coordinates
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
url

'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

In [1]:
# Individual Request

# params["location"] = f"-30.1811,-50.2056#"
# response = requests.get(url, params=params).json()
# results = response['results']   
# results

In [15]:
# Loop

for index, row in hotel_df.iterrows():
    vacation_latitude = row["City Latitude"]
    vacation_longitude = row["City Longitude"]
    
    params["location"] = f"{vacation_latitude},{vacation_longitude}"
    
    response = requests.get(url,params=params).json()
    
    results = response['results']
    
    try:
        hotel_df.loc[index,"Hotel Name"] = results[0]['name']
    except (IndexError,KeyError):
        hotel_df.loc[index,"Hotel Name"] = "There is no nearby Hotel"
    

In [16]:
hotel_df

,City ID,City Name,Country,City Latitude,City Longitude,City Temperature,City Max Temperature,City Humidity,City Cloudiness,City Wind Speed,Hotel Name
37,3430863,Mar del Plata,AR,-38.0023,-57.5575,73.40,73.40,60,0,6.91,Hotel Las Rocas
138,935215,Saint-Philippe,RE,-21.3585,55.7679,71.35,75.20,83,0,4.61,Les Embruns Du Baril
197,2063042,Port Hedland,AU,-20.3167,118.5667,71.60,71.60,73,0,3.44,Hedland Hotel
204,4033543,Teahupoo,PF,-17.8333,-149.2667,75.20,75.20,88,0,4.61,La Vague Bleu
214,3840300,Presidencia Roque Sáenz Peña,AR,-26.7852,-60.4388,71.01,71.01,53,0,1.99,Hotel Gualok
280,4034438,Haapiti,PF,-17.5667,-149.8667,75.20,75.20,88,0,4.61,MOOREA SURF INN
308,3437920,Itakyry,PY,-24.9333,-55.2167,73.40,73.40,46,0,6.91,There is no nearby Hotel
352,1529376,Korla,CN,41.7597,86.1469,73.58,73.58,15,0,3.18,Kangcheng Jian Guo International Hotel
360,1521379,Lenger,KZ,42.1830,69.8839,75.20,75.20,33,0,8.95,Gostevoy Dom 1
406,1185095,Teknāf,BD,20.8624,92.3058,79.23,79.23,83,0,1.41,Milky Resort


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["City Latitude", "City Longitude"]]

In [18]:
# Add marker layer ontop of heat map
hotel_markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map

fig.add_layer(hotel_markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))